In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import *
from keras import backend as K
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb 
from matplotlib import pyplot as plt

In [2]:
class attention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [3]:
fn = 'reshape_bb_2560'

In [4]:
df = pd.read_csv(r'../Data/BERT2560/Combined/'+ fn +'.csv')

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,3832,3833,3834,3835,3836,3837,3838,3839,label,uid
0,-0.579738,-0.317176,-0.834299,-0.439397,0.244214,-0.263808,-0.361836,0.331367,0.273019,-0.185394,...,0.295260,-0.032157,-0.182466,-0.200523,-0.383263,0.007856,0.218611,-0.005992,1.0,0
1,-0.465838,-0.231128,-0.654290,-0.523572,0.217441,-0.179935,-0.231489,0.001492,0.136614,-0.151429,...,0.319487,-0.153099,-0.469364,-0.284382,-0.425689,-0.006037,-0.054319,0.027232,0.0,1
2,-0.329293,-0.126982,-0.460519,-0.324948,-0.110048,-0.018931,-0.075261,0.287152,-0.023564,-0.082494,...,0.592590,-0.589239,0.061818,0.045413,0.066381,-0.360443,0.040208,0.385550,1.0,2
3,-0.295422,-0.207005,-0.680425,-0.586651,0.266593,-0.479852,-0.288612,0.314306,0.326187,-0.280874,...,0.420607,-0.455895,-0.294199,-0.327827,-0.255264,-0.034784,0.135940,0.318420,1.0,3
4,-0.508451,-0.188377,-0.161486,-0.389705,0.191571,-0.280671,-0.185133,0.130874,0.128341,-0.123900,...,0.168294,-0.110942,-0.378322,-0.317669,-0.334212,-0.222841,0.090401,0.175640,1.0,4


In [6]:
label = df['label']
df = df.drop(['uid', 'label'], axis = 1)

In [7]:
features = df

In [8]:
del df

In [9]:
features.shape

(39259, 3840)

In [11]:
features = features.to_numpy()
features = features.reshape(39259, 5, 768)

In [12]:
X, a, y, b = train_test_split(features, label, random_state=42, stratify=label, test_size=0.3)
print(X.shape, y.shape, a.shape, b.shape)

Xv, Xt, yv, yt = train_test_split(a, b, random_state=42, stratify=b, test_size=0.4)
print(Xv.shape, Xt.shape, yv.shape, yt.shape)

(27481, 5, 768) (27481,) (11778, 5, 768) (11778,)
(7066, 5, 768) (4712, 5, 768) (7066,) (4712,)


In [13]:
model = Sequential()
model.add(Bidirectional(LSTM(100, input_shape=(features.shape[1], features.shape[2]), return_sequences=True))) #bidirectional lstm
model.add(attention(return_sequences=True)) # receive 3D and output 2D
model.add(Dropout(0.25))
model.add(Bidirectional(LSTM(50, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(20)))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [15]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1000, patience=50)
#mc = ModelCheckpoint('best_model_legal_att' + fn + '_.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
history = model.fit(X, y, validation_data=(Xv, yv), epochs=1000, batch_size=16, callbacks=[es])

Epoch 1/1000
1718/1718 [==============================] - 33s 16ms/step - loss: 0.6760 - accuracy: 0.5700 - val_loss: 0.6723 - val_accuracy: 0.5756
Epoch 2/1000
1718/1718 [==============================] - 31s 18ms/step - loss: 0.6671 - accuracy: 0.5847 - val_loss: 0.6717 - val_accuracy: 0.5712
Epoch 3/1000
1718/1718 [==============================] - 28s 16ms/step - loss: 0.6596 - accuracy: 0.5967 - val_loss: 0.6615 - val_accuracy: 0.5965
Epoch 4/1000
1718/1718 [==============================] - 30s 18ms/step - loss: 0.6537 - accuracy: 0.6067 - val_loss: 0.6516 - val_accuracy: 0.6107
Epoch 5/1000
1718/1718 [==============================] - 34s 20ms/step - loss: 0.6466 - accuracy: 0.6170 - val_loss: 0.6529 - val_accuracy: 0.6040
Epoch 6/1000
1718/1718 [==============================] - 43s 25ms/step - loss: 0.6396 - accuracy: 0.6284 - val_loss: 0.6655 - val_accuracy: 0.6091
Epoch 7/1000
1718/1718 [==============================] - 40s 23ms/step - loss: 0.6324 - accuracy: 0.6376 - val_

In [ ]:
history_dict = history.history
print(history_dict.keys())

In [ ]:
saved_model = model
# evaluate the model
#_, train_acc = saved_model.evaluate(X, y, verbose=0)
#_, test_acc = saved_model.evaluate(Xt, yt, verbose=0)

#print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
lst = [train_acc, test_acc]
df = pd.DataFrame(lst, index =['train_acc', 'test_acc'])
df.to_csv(fn +'best_model_bb_att_results.csv')

In [ ]:
# plot history
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.savefig(fn + 'best_model_bb_att_acc.png', bbox_inches='tight')
pyplot.show()


In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
pyplot.savefig(fn + 'best_model_bb_att_loss.png', bbox_inches='tight')
plt.legend()
plt.show();